In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy

from statsmodels.sandbox.regression.gmm import GMM
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

In [2]:
data = pd.read_stata('chile.dta')
data = data.set_index(['id','year'])
print data['realmats'].min()

-36419.023


In [3]:
ldata = data.copy()

ldata[['routput', 'totlab', 'renerg','realmats',
       'rcapstock']] = np.log(data[['routput','totlab', 'renerg','realmats','rcapstock']])

ldata = ldata.dropna()

In [5]:
d = ldata['routput'],ldata[['renerg','totlab']],ldata[['rcapstock','realmats']]

def np_resids(y,x):
    """residuals from a 3 order polynomial"""
    y,x = y.dropna(),x.dropna()
    
    poly = PolynomialFeatures(degree=3)
    x_poly = poly.fit_transform(x)
    clf = Ridge(alpha=0.0)
    clf.fit(x_poly, y) 
    resid = y-clf.predict(x_poly)
    return resid

#print np_resids(ldata['routput'].shift(1),ldata[['rcapstock','realmats']].shift(1))

In [6]:
def robinson(dta):
    """y is dependent variable
    x has parametric coefficients
    z is nonparameteric variables"""
    
    y,x,z = dta
    
    y_resid = np_resids(y,z)
    x_resid = []
    for col in x.columns:
        resid = np_resids(x[col],z)
        x_resid.append(resid)
    x_resid = np.array(x_resid).transpose()
    x_resid = pd.DataFrame(data=x_resid,columns=x.columns)
    return sm.OLS(np.array(y_resid),x_resid)


d = ldata['routput'],ldata[['renerg','totlab']],ldata[['rcapstock','realmats']]
stage1 = robinson(d)
stage1_res = stage1.fit()
print stage1_res.summary()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.195
Model:                            OLS   Adj. R-squared:                  0.195
Method:                 Least Squares   F-statistic:                     2512.
Date:                Tue, 14 May 2019   Prob (F-statistic):               0.00
Time:                        14:55:40   Log-Likelihood:                -3246.9
No. Observations:               20717   AIC:                             6498.
Df Residuals:                   20715   BIC:                             6514.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
renerg         0.0858      0.002     39.327      0.0

In [15]:
def stage2obj(zparams, xparams, dta):
    """Note: z are the instruments in the stage 1 control function
    i.e. materials and capital
    
    x are the productive inputs who's effect was
    estimated in the first stage"""
    
    y,x,z = dta
    #########compute production trend and shock ##############
    shock1 = y - x.mul(xparams,axis=1).sum(axis=1) - z.mul(zparams,axis=1).sum(axis=1)

    ######  compute value added trend and shock ############
    #compute the value added by z + shock
    zvalue = y - x.mul(xparams,axis=1).sum(axis=1)
    
    #predict the value added by z
    zvalue_predict = zvalue - np_resids(zvalue, z)
    
    #predict the difference
    shock2 = zvalue_predict - z.mul(zparams,axis=1).sum(axis=1)

    #####compute expected production trend############
    shock2lag = shock2.groupby('id').shift(1)
    both_shocks = pd.concat((shock1,shock2lag),axis=1).dropna()
    trend = shock1 - np_resids(both_shocks[[0]],both_shocks[[1]])[0]
    
    ####subtract this trend from the shock ###########
    shock3 = shock1 - trend
    
    #####multiply by lags to get moment condition #########
    moments  = pd.concat((x,z),axis=1).groupby('id').shift(1)
    moments = moments.mul(shock3,axis=0) 
    return moments.dropna()
    
d = ldata['routput'],ldata[['renerg','totlab']],ldata[['rcapstock','realmats']]
stage2obj(np.ones(2), np.ones(2), d)

renerg     totlab  rcapstock   realmats
id       year                                            
18.0     1980   2.853471   1.179574   3.652708   3.427324
         1981  15.039531   6.763259  21.733869  21.445536
54.0     1980  -9.166693  -4.069390  -9.819149 -11.191349
         1981  -3.643825  -1.585153  -3.761476  -4.457207
         1982  -7.373051  -2.902532  -7.824633  -8.749781
         1983   2.946377   0.999427   3.266438   3.709128
         1984   1.732807   0.611860   2.096807   2.237856
         1985   4.132891   1.516244   5.076901   5.591079
         1986   1.635285   0.756980   2.188037   2.450566
         1987   0.362574   0.136276   0.388328   0.450300
126.0    1989 -27.701355 -16.250158 -22.021133 -34.198778
         1990 -30.890124 -16.941940 -32.174402 -37.744927
         1991  -9.533793  -6.403578 -10.073803 -11.590913
         1992 -11.804703  -6.818126 -13.270819 -15.140869
         1993 -10.060634  -5.604982 -10.928342 -12.446265
         1994 -19.188530 -10.028573 -20.216876 -21.807760
         1995 -20.018940 -10.538659 -20.934753 -23.396268
504.0    1980  -9.063435  -4.953284 -12.870538 -15.581066
         1981  -5.826625  -1.836463  -5.479936  -6.545643
         1982 -15.762838  -6.261140 -16.121481 -19.250728
         1983  -2.404015  -0.930738  -2.163347  -2.659569
513.0    1982  -1.137978  -0.388426  -0.595240  -1.341142
         1983 -25.372283  -9.970089 -15.178228 -29.875194
         1984   1.734283   0.656698   2.045863   1.828119
         1985   1.091082   0.455882   1.566637   1.148021
         1986  10.049701   4.403569  15.049075  10.770465
         1987  -7.578557  -2.497862 -11.843117  -7.828111
         1988  -5.059077  -2.080188  -6.499701  -4.636284
522.0    1980  -2.241652  -0.922153  -2.618561  -2.845701
         1981  -5.921082  -1.999033  -6.589801  -7.202555
...                  ...        ...        ...        ...
251235.0 1996  -4.214578  -2.184103  -5.339939  -5.688472
251388.0 1996  11.080768   5.432423  15.590605  14.581163
251433.0 1996   0.526579   0.218683   0.625091   0.697548
251478.0 1996   7.332696   3.760555   9.664247  10.766485
251676.0 1996   0.381656   0.205959   0.474173   0.566804
251712.0 1996   7.565797   2.658486   8.991820   9.760075
251748.0 1996  -7.647714  -3.235328  -9.875685  -9.533099
251829.0 1996 -16.971417  -6.982182 -21.370349 -21.450587
251838.0 1996 -10.312590  -3.829388 -10.066608 -13.275674
251847.0 1996  -0.060871  -0.029159  -0.043327  -0.075835
251874.0 1996  -4.177931  -2.038907  -4.910527  -5.449554
251883.0 1996  -7.746482  -3.009079  -7.974801  -9.663661
252081.0 1996  -1.448287  -0.658880  -1.400073  -2.192439
252207.0 1996  21.010277   9.225800  23.592242  28.254720
252315.0 1996  -1.535588  -0.608441  -1.636148  -2.059899
252324.0 1996 -20.425870  -6.777755 -20.596601 -28.379869
252513.0 1996   5.462259   2.888858   7.298045   8.062570
252666.0 1996  -8.048736  -2.514721  -8.111191 -10.409086
252882.0 1996 -18.453286  -7.259556 -21.418119 -20.887807
252918.0 1996   5.731960   2.697627   7.679639   7.726658
252963.0 1996 -23.921755  -8.524477 -21.138745 -26.161025
252972.0 1996  -2.173291  -0.986573  -2.663002  -2.856914
252981.0 1996  11.521231   4.608484  15.286575  12.536120
253215.0 1996  -7.476278  -2.938889  -8.101869  -8.495744
253224.0 1996 -16.823092  -7.216634 -16.832016 -22.113035
253269.0 1996  -5.904213  -2.871868  -7.581439  -7.189791
253341.0 1996  -3.668044  -1.272420  -4.315614  -4.452986
253431.0 1996  -2.856184  -1.196953  -3.677033  -3.841933
253593.0 1996   7.770006   3.184533  12.820820  11.947485
253629.0 1996 -31.092660 -18.570068 -37.134803 -42.301742

[18044 rows x 4 columns]

In [16]:
class LP(GMM):
    
    def __init__(self, dta, stage1_params, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        y,x,z = dta
        super(LP, self).__init__(y,x,z, x.shape[1] +z.shape[1],*args, **kwds)
        self.endog = y
        self.exog = x
        self.instr = z
        self.stage1_params = stage1_params
        
        self.data.xnames = [col for col in z.columns]
        
    def momcond(self, params):
        d = self.endog, self.exog, self.instr
        return np.array(stage2obj(params, self.stage1_params , d))
        
        
d = ldata['routput'],ldata[['renerg','totlab']],ldata[['rcapstock','realmats']]
lpmodel = LP(d,stage1_res.params)
lpresult = lpmodel.fit(np.array([.1,.5]),optim_method='nm')
print lpresult.summary()

Optimization terminated successfully.
         Current function value: 0.000147
         Iterations: 27
         Function evaluations: 53
Optimization terminated successfully.
         Current function value: 0.000842
         Iterations: 28
         Function evaluations: 53
Optimization terminated successfully.
         Current function value: 0.000842
         Iterations: 18
         Function evaluations: 35
Optimization terminated successfully.
         Current function value: 0.000842
         Iterations: 19
         Function evaluations: 36
                                  LP Results                                  
Dep. Variable:                routput   Hansen J:                        15.20
Model:                             LP   Prob (Hansen J):              0.000501
Method:                           GMM                                         
Date:                Tue, 14 May 2019                                         
Time:                        14:58:33                